# Which cards are most commonly used by top players?
This approach preprocesses the dataset by combining the deck1 and deck2 columns into a single column and converting it into a document-term matrix using CountVectorizer. It then trains an LDA model on the document-term matrix to identify topics, and identifies the most common cards associated with each topic. The top cards associated with the most frequent topics can be considered the most commonly used by top players. Note that this is just an example approach, and there may be other factors that contribute to card usage besides the top players.

In [27]:
!pip install mlxtend
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from mlxtend.frequent_patterns import apriori, association_rules
import ast

In [28]:
df=pd.read_csv("../Common/Final Cleaned/battle_log_cleaned.csv")
df['deck1'] = df['deck1'].apply(ast.literal_eval)
df['deck2'] = df['deck2'].apply(ast.literal_eval)
df.head(2)

,type,gameMode,crowns1,startingTrophies2,crowns2,deck1,deck2,max_level_1,level_1,max_level_2,level_2,avg_cards_level_1,avg_cards_level_2,avg_cards_level_diff,1_won,2_won
0,PvP,"{'id': 72000201, 'name': 'Ladder_CrownRush'}",0,5807.0,3,"[Arrows, Bomber, Electro Wizard, Skeleton Army...","[Magic Archer, Mega Knight, Electro Wizard, Go...","[14, 14, 6, 9, 14, 12, 12, 6]","[11, 12, 2, 5, 14, 12, 10, 3]","[6, 6, 6, 9, 6, 9, 14, 12]","[4, 6, 4, 8, 3, 9, 13, 12]",11.75,12.875,1.125,False,True
1,PvP,"{'id': 72000201, 'name': 'Ladder_CrownRush'}",0,5857.0,2,"[Arrows, Bomber, Electro Wizard, Skeleton Army...","[Mini P.E.K.K.A, Wall Breakers, Valkyrie, Gobl...","[14, 14, 6, 9, 14, 12, 12, 6]","[11, 12, 2, 5, 14, 12, 10, 3]","[12, 9, 12, 9, 12, 9, 14, 6]","[12, 7, 10, 8, 10, 9, 12, 4]",11.75,12.625,0.875,False,True


In [29]:
mlb = MultiLabelBinarizer()
card_matrix = mlb.fit_transform(df["deck1"])


In [30]:
high_trophy_players = df[df["startingTrophies2"] > df["startingTrophies2"].quantile(0.75)]
card_df = pd.DataFrame(card_matrix, columns=mlb.classes_)

frequent_itemsets = apriori(card_df.loc[high_trophy_players.index], min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.sort_values(by=["lift", "support"], ascending=[False, False])


C:\Users\arthu\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [31]:
most_common_cards = card_df.loc[high_trophy_players.index].mean().sort_values(ascending=False).head(10)
most_common_itemsets = rules[["antecedents", "consequents", "support", "lift"]].head(10)


In [33]:
most_common_cards = card_df.loc[high_trophy_players.index].mean().sort_values(ascending=False).head(10)
most_common_itemsets = rules[["antecedents", "consequents", "support", "lift"]].head(10)

print("Most commonly used individual cards by top players:")
print(most_common_cards)

print("\nMost commonly used card combinations by top players:")
print(most_common_itemsets)


Most commonly used individual cards by top players:
Valkyrie         0.375627
Hog Rider        0.359429
The Log          0.347860
Zap              0.315850
Fireball         0.280756
Arrows           0.267644
Mega Knight      0.264558
Wizard           0.236020
Goblin Barrel    0.224450
Skeleton Army    0.223679
dtype: float64

Most commonly used card combinations by top players:
        antecedents      consequents   support      lift
4     (Mega Knight)           (Bats)  0.107212  2.235761
5            (Bats)    (Mega Knight)  0.107212  2.235761
6        (Fireball)      (Hog Rider)  0.139221  1.379629
7       (Hog Rider)       (Fireball)  0.139221  1.379629
10        (The Log)  (Goblin Barrel)  0.102970  1.318816
11  (Goblin Barrel)        (The Log)  0.102970  1.318816
18       (Valkyrie)      (Hog Rider)  0.177015  1.311114
19      (Hog Rider)       (Valkyrie)  0.177015  1.311114
12       (Valkyrie)  (Goblin Barrel)  0.102584  1.216752
13  (Goblin Barrel)       (Valkyrie)  0.102584  1

## Conclusion

Based on the analysis, we can conclude that Valkyrie, Hog Rider, and The Log are the three most commonly used individual cards by top players. In terms of card combinations, Mega Knight and Bats, Bats and Mega Knight, Fireball and Hog Rider, Hog Rider and Fireball, and The Log and Goblin Barrel are among the most commonly used card combinations by top players. These insights can be used to inform gameplay strategy, as players can consider using these cards and combinations in their own decks to improve their chances of winning battles.